In [1]:
Week1_Water_Quality_Prediction:

In [ ]:
# Import necessary libraries
import pandas as pd # data manipulation
import numpy as np # numerical python - linear algebra

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# load the dataset
df = pd.read_csv('afa2e701598d20110228.csv', sep=';')
df

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,17.02.2000,0.330,2.77,12.0,12.30,9.50,0.057,154.00,0.454,289.50
1,1,11.05.2000,0.044,3.00,51.6,14.61,17.75,0.034,352.00,0.090,1792.00
2,1,11.09.2000,0.032,2.10,24.5,9.87,13.80,0.173,416.00,0.200,2509.00
3,1,13.12.2000,0.170,2.23,35.6,12.40,17.13,0.099,275.20,0.377,1264.00
4,1,02.03.2001,0.000,3.03,48.8,14.69,10.00,0.065,281.60,0.134,1462.00
...,...,...,...,...,...,...,...,...,...,...,...
2856,22,06.10.2020,0.046,2.69,3.6,8.28,3.80,0.038,160.00,0.726,77.85
2857,22,27.10.2020,0.000,1.52,0.5,11.26,0.56,0.031,147.20,0.634,71.95
2858,22,03.12.2020,0.034,0.29,0.8,11.09,2.58,0.042,209.92,0.484,61.17
2859,22,12.01.2021,0.000,2.10,0.0,14.31,3.94,0.034,121.60,0.424,63.49


In [3]:
# Check and handle missing values
missing_values = df.isnull().sum()
print("Missing values before cleaning:\n", missing_values)

Missing values before cleaning:
 id            0
date          0
NH4           3
BSK5          1
Suspended    16
O2            3
NO3           1
NO2           3
SO4          49
PO4          28
CL           49
dtype: int64


In [4]:
# Confirm all missing values are handled
print("\nMissing values after cleaning:\n", df.isnull().sum())


Missing values after cleaning:
 id            0
date          0
NH4           3
BSK5          1
Suspended    16
O2            3
NO3           1
NO2           3
SO4          49
PO4          28
CL           49
dtype: int64


In [5]:
# Display column names to confirm structure
print("Columns in dataset:\n", df.columns)

Columns in dataset:
 Index(['id', 'date', 'NH4', 'BSK5', 'Suspended', 'O2', 'NO3', 'NO2', 'SO4',
       'PO4', 'CL'],
      dtype='object')


In [6]:
#Show cleaned dataset shape and first few rows
print("\nDataset shape:", df.shape)
print(df.head())


Dataset shape: (2861, 11)
   id        date    NH4  BSK5  Suspended     O2    NO3    NO2    SO4    PO4  \
0   1  17.02.2000  0.330  2.77       12.0  12.30   9.50  0.057  154.0  0.454   
1   1  11.05.2000  0.044  3.00       51.6  14.61  17.75  0.034  352.0  0.090   
2   1  11.09.2000  0.032  2.10       24.5   9.87  13.80  0.173  416.0  0.200   
3   1  13.12.2000  0.170  2.23       35.6  12.40  17.13  0.099  275.2  0.377   
4   1  02.03.2001  0.000  3.03       48.8  14.69  10.00  0.065  281.6  0.134   

       CL  
0   289.5  
1  1792.0  
2  2509.0  
3  1264.0  
4  1462.0  


In [7]:
# Check columns
print("Columns:", df.columns.tolist())

Columns: ['id', 'date', 'NH4', 'BSK5', 'Suspended', 'O2', 'NO3', 'NO2', 'SO4', 'PO4', 'CL']


In [8]:
df.info() # dataset info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         2861 non-null   int64  
 1   date       2861 non-null   object 
 2   NH4        2858 non-null   float64
 3   BSK5       2860 non-null   float64
 4   Suspended  2845 non-null   float64
 5   O2         2858 non-null   float64
 6   NO3        2860 non-null   float64
 7   NO2        2858 non-null   float64
 8   SO4        2812 non-null   float64
 9   PO4        2833 non-null   float64
 10  CL         2812 non-null   float64
dtypes: float64(9), int64(1), object(1)
memory usage: 246.0+ KB


In [9]:
# Statistics of the data
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,2861.0,12.397064,6.084226,1.00,8.0000,14.000,16.00000,22.000
NH4,2858.0,0.758734,2.486247,0.00,0.0800,0.220,0.50000,39.427
BSK5,2860.0,4.316182,2.973997,0.00,2.1600,3.800,5.80000,50.900
Suspended,2845.0,12.931905,16.543097,0.00,6.0000,10.000,15.00000,595.000
O2,2858.0,9.508902,4.428260,0.00,7.0925,8.995,11.52000,90.000
NO3,2860.0,4.316846,6.881188,0.00,1.3900,2.800,5.58250,133.400
NO2,2858.0,0.246128,2.182777,0.00,0.0300,0.059,0.12575,109.000
SO4,2812.0,59.362313,96.582641,0.00,27.0525,37.800,64.64000,3573.400
PO4,2833.0,0.418626,0.771326,0.00,0.1300,0.270,0.47000,13.879
CL,2812.0,93.731991,394.512184,0.02,26.8000,33.900,45.60750,5615.280


In [10]:
# Missing values
df.isnull().sum()

id            0
date          0
NH4           3
BSK5          1
Suspended    16
O2            3
NO3           1
NO2           3
SO4          49
PO4          28
CL           49
dtype: int64

In [11]:
# rows and cols
df.shape


(2861, 11)

In [12]:
# date is in object - date format
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,2000-02-17,0.330,2.77,12.0,12.30,9.50,0.057,154.00,0.454,289.50
1,1,2000-05-11,0.044,3.00,51.6,14.61,17.75,0.034,352.00,0.090,1792.00
2,1,2000-09-11,0.032,2.10,24.5,9.87,13.80,0.173,416.00,0.200,2509.00
3,1,2000-12-13,0.170,2.23,35.6,12.40,17.13,0.099,275.20,0.377,1264.00
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.00,0.065,281.60,0.134,1462.00
...,...,...,...,...,...,...,...,...,...,...,...
2856,22,2020-10-06,0.046,2.69,3.6,8.28,3.80,0.038,160.00,0.726,77.85
2857,22,2020-10-27,0.000,1.52,0.5,11.26,0.56,0.031,147.20,0.634,71.95
2858,22,2020-12-03,0.034,0.29,0.8,11.09,2.58,0.042,209.92,0.484,61.17
2859,22,2021-01-12,0.000,2.10,0.0,14.31,3.94,0.034,121.60,0.424,63.49


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         2861 non-null   int64         
 1   date       2861 non-null   datetime64[ns]
 2   NH4        2858 non-null   float64       
 3   BSK5       2860 non-null   float64       
 4   Suspended  2845 non-null   float64       
 5   O2         2858 non-null   float64       
 6   NO3        2860 non-null   float64       
 7   NO2        2858 non-null   float64       
 8   SO4        2812 non-null   float64       
 9   PO4        2833 non-null   float64       
 10  CL         2812 non-null   float64       
dtypes: datetime64[ns](1), float64(9), int64(1)
memory usage: 246.0 KB


In [14]:
df.describe()

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
count,2861.000000,2861,2858.000000,2860.000000,2845.000000,2858.000000,2860.000000,2858.000000,2812.000000,2833.000000,2812.000000
mean,12.397064,2010-05-02 22:53:33.701502976,0.758734,4.316182,12.931905,9.508902,4.316846,0.246128,59.362313,0.418626,93.731991
min,1.000000,2000-01-10 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000
25%,8.000000,2005-03-10 00:00:00,0.080000,2.160000,6.000000,7.092500,1.390000,0.030000,27.052500,0.130000,26.800000
50%,14.000000,2010-06-04 00:00:00,0.220000,3.800000,10.000000,8.995000,2.800000,0.059000,37.800000,0.270000,33.900000
75%,16.000000,2015-08-11 00:00:00,0.500000,5.800000,15.000000,11.520000,5.582500,0.125750,64.640000,0.470000,45.607500
max,22.000000,2021-04-07 00:00:00,39.427000,50.900000,595.000000,90.000000,133.400000,109.000000,3573.400000,13.879000,5615.280000
std,6.084226,NaN,2.486247,2.973997,16.543097,4.428260,6.881188,2.182777,96.582641,0.771326,394.512184


In [15]:
df = df.sort_values(by=['id', 'date'])
df.head()


,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,2000-02-17,0.330,2.77,12.0,12.30,9.50,0.057,154.0,0.454,289.5
1,1,2000-05-11,0.044,3.00,51.6,14.61,17.75,0.034,352.0,0.090,1792.0
2,1,2000-09-11,0.032,2.10,24.5,9.87,13.80,0.173,416.0,0.200,2509.0
3,1,2000-12-13,0.170,2.23,35.6,12.40,17.13,0.099,275.2,0.377,1264.0
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.00,0.065,281.6,0.134,1462.0


In [16]:
# Missing values per column
print("Missing Values:\n", df.isnull().sum())

# Data types of each column
print("\nData Types:\n", df.dtypes)

Missing Values:
 id            0
date          0
NH4           3
BSK5          1
Suspended    16
O2            3
NO3           1
NO2           3
SO4          49
PO4          28
CL           49
dtype: int64

Data Types:
 id                    int64
date         datetime64[ns]
NH4                 float64
BSK5                float64
Suspended           float64
O2                  float64
NO3                 float64
NO2                 float64
SO4                 float64
PO4                 float64
CL                  float64
dtype: object


In [19]:
# Get correlation matrix
corr_matrix = df.corr(numeric_only=True)
print("\nCorrelation Matrix:\n", corr_matrix)


Correlation Matrix:
                  id       NH4      BSK5  Suspended        O2       NO3  \
id         1.000000  0.197706  0.176752   0.019696 -0.112548 -0.118513   
NH4        0.197706  1.000000  0.266770   0.150009 -0.149066  0.030816   
BSK5       0.176752  0.266770  1.000000   0.091744 -0.118188 -0.015250   
Suspended  0.019696  0.150009  0.091744   1.000000 -0.014197  0.161389   
O2        -0.112548 -0.149066 -0.118188  -0.014197  1.000000  0.110719   
NO3       -0.118513  0.030816 -0.015250   0.161389  0.110719  1.000000   
NO2        0.016002  0.079176  0.122532   0.048427  0.235794  0.033689   
SO4       -0.203685 -0.052079 -0.130125   0.212503  0.305596  0.311238   
PO4        0.112606  0.600082  0.196032   0.087346 -0.182137 -0.005295   
CL        -0.267060 -0.031530 -0.019696   0.389624  0.058526  0.437123   

                NO2       SO4           PO4            CL  
id         0.016002 -0.203685  1.126064e-01 -2.670595e-01  
NH4        0.079176 -0.052079  6.000821e-01

In [20]:
# Skewness (asymmetry)
print("Skewness:\n", df.skew(numeric_only=True))

# Kurtosis (peakedness)
print("\nKurtosis:\n", df.kurt(numeric_only=True))

Skewness:
 id           -0.226620
NH4           8.398825
BSK5          3.040943
Suspended    16.870337
O2            6.037018
NO3          10.080634
NO2          43.861030
SO4          19.157920
PO4           9.376081
CL            8.194800
dtype: float64

Kurtosis:
 id             -0.906550
NH4            86.314258
BSK5           28.556878
Suspended     545.206672
O2             87.506356
NO3           148.977191
NO2          2161.344292
SO4           632.297548
PO4           119.792440
CL             75.234811
dtype: float64


In [ ]:
# Unique values in each column
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

# Min/Max for selected features
print("\nNO3 range:", df['no3'].min(), "to", df['no3'].max())
print("O2 range:", df['o2'].min(), "to", df['o2'].max())

In [ ]:
Week2_Water_Quality_Prediction:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [5]:
# load the dataset
df = pd.read_csv('afa2e701598d20110228.csv', sep=';')
df

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,17.02.2000,0.330,2.77,12.0,12.30,9.50,0.057,154.00,0.454,289.50
1,1,11.05.2000,0.044,3.00,51.6,14.61,17.75,0.034,352.00,0.090,1792.00
2,1,11.09.2000,0.032,2.10,24.5,9.87,13.80,0.173,416.00,0.200,2509.00
3,1,13.12.2000,0.170,2.23,35.6,12.40,17.13,0.099,275.20,0.377,1264.00
4,1,02.03.2001,0.000,3.03,48.8,14.69,10.00,0.065,281.60,0.134,1462.00
...,...,...,...,...,...,...,...,...,...,...,...
2856,22,06.10.2020,0.046,2.69,3.6,8.28,3.80,0.038,160.00,0.726,77.85
2857,22,27.10.2020,0.000,1.52,0.5,11.26,0.56,0.031,147.20,0.634,71.95
2858,22,03.12.2020,0.034,0.29,0.8,11.09,2.58,0.042,209.92,0.484,61.17
2859,22,12.01.2021,0.000,2.10,0.0,14.31,3.94,0.034,121.60,0.424,63.49


In [9]:
# drop the missing values - dropna()
df = df.dropna(subset=pollutants)
df.head()


,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL,year
0,1,17.02.2000,0.330,2.77,12.0,12.30,9.50,0.057,154.0,0.454,289.5,2000
1,1,11.05.2000,0.044,3.00,51.6,14.61,17.75,0.034,352.0,0.090,1792.0,2000
2,1,11.09.2000,0.032,2.10,24.5,9.87,13.80,0.173,416.0,0.200,2509.0,2000
3,1,13.12.2000,0.170,2.23,35.6,12.40,17.13,0.099,275.2,0.377,1264.0,2000
4,1,02.03.2001,0.000,3.03,48.8,14.69,10.00,0.065,281.6,0.134,1462.0,2001


In [24]:
df.isnull().sum()

id           0
date         0
NH4          0
BSK5         0
Suspended    0
O2           0
NO3          0
NO2          0
SO4          0
PO4          0
CL           0
year         0
dtype: int64

In [25]:
#  Feature and Target Selection
# Features (independent variables): 'id' and 'year'
# Target (dependent variables): pollutant concentrations
features = ['id', 'year']
X = df[features]
y = df[pollutants]

In [ ]:
# Encoding Categorical Feature 'id' using One-Hot Encoding
# drop_first=True avoids dummy variable trap by removing one column
X_encoded = pd.get_dummies(X, columns=['id'], prefix='station', drop_first=True)

In [ ]:
# Optional: ensure consistent column order
X_encoded = X_encoded.sort_index(axis=1)


In [26]:
# Model configuration
base_model = RandomForestRegressor(
    n_estimators=150,       # Slightly increased for better performance
    max_depth=None,         # Can set to a value like 10 for faster training (optional)
    min_samples_split=4,    # Prevents overfitting slightly
    random_state=42,
    n_jobs=-1               # Enables parallel training on all cores
)

# Multi-output model for all pollutants
model = MultiOutputRegressor(base_model)
model.fit(X_train, y_train)


MultiOutputRegressor(estimator=RandomForestRegressor(min_samples_split=4,
                                                     n_estimators=150,
                                                     n_jobs=-1,
                                                     random_state=42))

In [17]:
# Evaluate
print("Model Performance on the Test Data:\n")
for i, pollutant in enumerate(pollutants):
    mse = mean_squared_error(y_test.iloc[:, i], y_pred[:, i])
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    print(f'{pollutant}:')
    print(f'   MSE: {mse:.2f}')
    print(f'   R²: {r2:.2f}\n')


Model Performance on the Test Data:

NH4:
   MSE: 0.88
   R²: 0.78

BSK5:
   MSE: 5.31
   R²: 0.19

Suspended:
   MSE: 98.18
   R²: 0.20

O2:
   MSE: 13.96
   R²: 0.05

NO3:
   MSE: 20.40
   R²: 0.48

NO2:
   MSE: 10.34
   R²: -58.20

SO4:
   MSE: 2275.81
   R²: 0.45

PO4:
   MSE: 0.24
   R²: 0.44

CL:
   MSE: 32661.44
   R²: 0.75



In [11]:
# Evaluate model
y_pred = model.predict(X_test)
print("Model Performance on the Test Data:")
for i, pollutant in enumerate(pollutants):
    print(f'{pollutant}:')
    print('   MSE:', mean_squared_error(y_test.iloc[:, i], y_pred[:, i]))
    print('   R2:', r2_score(y_test.iloc[:, i], y_pred[:, i]))
    print()

Model Performance on the Test Data:
NH4:
   MSE: 0.8827195364614927
   R2: 0.7801981883484587

BSK5:
   MSE: 5.31094542545559
   R2: 0.19096990064204467

Suspended:
   MSE: 98.17784721522588
   R2: 0.20495839046561737

O2:
   MSE: 13.955930601011778
   R2: 0.05381534726017545

NO3:
   MSE: 20.40490374797047
   R2: 0.484569230962687

NO2:
   MSE: 10.343405404494533
   R2: -58.203860061465534

SO4:
   MSE: 2275.807351900022
   R2: 0.44815941114800695

PO4:
   MSE: 0.24389334027446746
   R2: 0.43586964570072984

CL:
   MSE: 32661.43741785968
   R2: 0.7526035914013255



In [12]:
station_id = '22'
year_input = 2024

input_data = pd.DataFrame({'year': [year_input], 'id': [station_id]})
input_encoded = pd.get_dummies(input_data, columns=['id'])

# Align with training feature columns
missing_cols = set(X_encoded.columns) - set(input_encoded.columns)
for col in missing_cols:
    input_encoded[col] = 0
input_encoded = input_encoded[X_encoded.columns]  # reorder columns

# Predict pollutants
predicted_pollutants = model.predict(input_encoded)[0]

print(f"\nPredicted pollutant levels for station '{station_id}' in {year_input}:")
for p, val in zip(pollutants, predicted_pollutants):
    print(f"  {p}: {val:.2f}")


Predicted pollutant levels for station '22' in 2024:
  NH4: 0.03
  BSK5: 2.57
  Suspended: 5.69
  O2: 13.25
  NO3: 6.93
  NO2: 0.07
  SO4: 144.84
  PO4: 0.46
  CL: 67.36


In [20]:
# Function to prepare features and targets
def encode_features(df):
    X = df[['id', 'year']]
    y = df[pollutants]
    X_encoded = pd.get_dummies(X, columns=['id'], drop_first=True)
    return X_encoded, y

In [15]:
# Make prediction
predicted_pollutants = model.predict(input_encoded)[0]
print(f"\nPredicted pollutant levels for station '{station_id}' in {year_input}:")
for p, val in zip(pollutants, predicted_pollutants):
    print(f"  {p}: {val:.2f}")


Predicted pollutant levels for station '22' in 2024:
  NH4: 0.03
  BSK5: 2.57
  Suspended: 5.69
  O2: 13.25
  NO3: 6.93
  NO2: 0.07
  SO4: 144.84
  PO4: 0.46
  CL: 67.36


In [ ]:
# Function to load and preprocess the dataset
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=';')
    df['year'] = pd.to_datetime(df['date'], dayfirst=True).dt.year
    df = df.dropna(subset=pollutants)
    return df

In [ ]:
# Function to prepare features and targets
def encode_features(df):
    X = df[['id', 'year']]
    y = df[pollutants]
    X_encoded = pd.get_dummies(X, columns=['id'], drop_first=True)
    return X_encoded, y

In [ ]:
# Function to predict pollutant levels for a new station-year
def predict_pollutants(model, X_columns, station_id, year):
    input_df = pd.DataFrame({'year': [year], 'id': [station_id]})
    input_encoded = pd.get_dummies(input_df, columns=['id'])


In [28]:
import joblib

joblib.dump(model, 'pollution_model.pkl')
joblib.dump(X_encoded.columns.tolist(), "model_columns.pkl")
print('Model and cols structure are saved!')

Model and cols structure are saved!


In [29]:
import joblib

# Define file paths
model_filename = "pollution_rf_model.pkl"
columns_filename = "pollution_feature_columns.pkl"

# Save the trained model and feature column structure
joblib.dump(model, model_filename)
joblib.dump(X_encoded.columns.tolist(), columns_filename)

# Confirmation message
print(f" Model saved as: {model_filename}")
print(f" Feature columns saved as: {columns_filename}")


 Model saved as: pollution_rf_model.pkl
 Feature columns saved as: pollution_feature_columns.pkl
